# Data Cleaning for Geo IDs

My data sets are reported in different geographical units, ZIP codes, city names, county, state, and census geographic codes. I created this data set (geo_df) to match each other. Geo IDs match zip code - county IDs (county fips) - city ID - state ID. It also has the latitude and longitude of each zip code. I can use this for visualization with geo pandas. 

In this notebook, I go through the process of creating Geo IDs data frame.

Contents:
1. Matching ZIP codes to census geographical IDs 
2. Matching ZIP codes to Economic Tracker IDs 
3. Matching ZIP codes to latitude and longitude data


In [1]:
import requests

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")


# 1. Matching ZIP codes to census geographical IDs  

First, I match ZIP code to all census geographical IDs. For this, I use the data from this webside. https://www.huduser.gov/portal/datasets/usps_crosswalk.html  
The difficulty of matching ZIP codes to census IDs is finding the one-to-one matching. Some zip codes appear in different counties and cities. In this case, I kept the most commonly used county/city name to match ZIP code and county.

First, I read the data downloaded from https://www.huduser.gov/portal/datasets/usps_crosswalk.html

In [2]:
# Read data 
cbsa = pd.read_excel('Data/Crosswalk/ZIP_CBSA_092021.xlsx')
cbsadv = pd.read_excel('Data/Crosswalk/ZIP_CBSA_DIV_092021.xlsx')
cd = pd.read_excel('Data/Crosswalk/ZIP_CD_092021.xlsx') 
county = pd.read_excel('Data/Crosswalk/ZIP_COUNTY_092021.xlsx') 

# Drop unnecessary columns 
cbsa = cbsa.iloc[:, 0:4]
cbsadv = cbsadv.iloc[:, 0:4]
cd = cd.iloc[:, 0:4]
#county = county

# merge 
#df = county.merge(cbsadv, how='outer', on='ZIP')
#df = df.merge(cd, how='outer', on='ZIP')
#df = df.merge(cbsa, how='outer', on='ZIP')

#df.drop_duplicates()

If a zip code has multiple county names, I keep the county name that has the most appearance using 'total ratio.'

In [3]:
# If a zipcode has multiple county names, keep the county name which has the highest total ratio. 

# Add a column which return the maximum tot_ratio in a same zipcode  
max_tot =  pd.DataFrame(county.groupby('ZIP').max()['TOT_RATIO'])
max_tot.reset_index(inplace=True)
county = county.merge(max_tot, on='ZIP', how='left')

# if total ratio is smaller than max, drop the zipcode. 
county = county[county.TOT_RATIO_x == county.TOT_RATIO_y]

Some zip codes still have multiple county names because tot_ratio is 50:50 split. In this case, I use RES_RATIO to assign county name. 

In [4]:
# If tot_ratio is 50:50 split, use RES_RATIO to assign county name. 

# Add a column which return the maximum RES_ratio in a same zipcode  
max_res =  pd.DataFrame(county.groupby('ZIP').max()['RES_RATIO'])
max_res.reset_index(inplace=True)
county = county.merge(max_res, on='ZIP', how='left')

# if Res ratio is smaller than max, drop the zipcode. 
county = county[county.RES_RATIO_x == county.RES_RATIO_y]


Lastly, if tot_ratio and RES_RATIO are 50:50 split, I use BUS_RATIO to assign county name. 

In [5]:
# If tot_ratio and RES_RATIO is 50:50 split, use BUS_RATIO to assign county name. 

# Add a column which return the maximum RES_ratio in a same zipcode  
max_bus =  pd.DataFrame(county.groupby('ZIP').max()['BUS_RATIO'])
max_bus.reset_index(inplace=True)
county = county.merge(max_bus, on='ZIP', how='left')

# if Res ratio is smaller than max, drop the zipcode. 
county = county[county.BUS_RATIO_x == county.BUS_RATIO_y]

# Lastly, for ZIP code=51603, drop the second entry of county.   
county.drop(county[(county.ZIP==51603) & (county.COUNTY==19071)].index, inplace = True)

In [6]:
county.nunique()

ZIP                    39488
COUNTY                  3226
USPS_ZIP_PREF_CITY     18506
USPS_ZIP_PREF_STATE       56
RES_RATIO_x            10206
BUS_RATIO_x             3318
OTH_RATIO               1703
TOT_RATIO_x            10401
TOT_RATIO_y            10401
RES_RATIO_y            10206
BUS_RATIO_y             3318
dtype: int64

In [7]:
county.shape

(39488, 11)

Now, I created a dataset that uniquely identifies 39,488 zip codes to Census geographical codes.

# 2. Matching ZIP codes to Economic Tracker IDs

Next, I match ZIP codes to Economic Tracker geographical IDs. I use economic data from Economic Tracker data base. So, I merge their geographical IDs to ZIP codes. I donwloaded GeoID-County data from Economic Tracker at https://github.com/OpportunityInsights/EconomicTracker/tree/main/data 

In [8]:
# Merge GeoIDs-County to zip_city_new using countyfips 

# Downloard GeoID-County data from Economic Tracker at https://github.com/OpportunityInsights/EconomicTracker/tree/main/data 
# Read the file
geoid_county = pd.read_csv('Data/EconomicTracker-main/data/GeoIDs - County.csv')

# merge geoid_county to county
county_zip = geoid_county.merge(county, how='right', left_on='countyfips', right_on='COUNTY')

#drop if zip codes are missing. 
county_zip.dropna(axis=0, subset=['ZIP'])


county_zip.head()

,countyfips,countyname,cityid,cityname,cz,czname,statename,statefips,stateabbrev,county_pop2019,...,COUNTY,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO,TOT_RATIO_x,TOT_RATIO_y,RES_RATIO_y,BUS_RATIO_y
0,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,36103,HOLTSVILLE,NY,0.000000,1.0,0.0,1.000000,1.000000,0.000000,1.0
1,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,36103,FISHERS ISLAND,NY,0.000000,1.0,1.0,1.000000,1.000000,0.000000,1.0
2,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,36103,AMITYVILLE,NY,0.994811,1.0,1.0,0.995644,0.995644,0.994811,1.0
3,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,36103,BABYLON,NY,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0
4,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,36103,NORTH BABYLON,NY,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0


In the cell below, I assign the nearest city name and its lon&lat for each zip code located in a metropolitan area. For this, I used GeoID -City file from Econmic Tracker data set. For each zipcode, I check the czname (metropolitan area name). I use the  GeoID -City file to find the nearest city for each czname. 


In [9]:
# Use cities included in the GEOID-City for a city name. 
# For each zipcode in county_zip file, check the czname (economic zone named by the nearest city). If czname is in the GEOID-City file, assine the nearest city. 
# I also add the nearest city's latitude and longitude from GEOID-City 

# Read the GeoID -City file and name it czone 
czone = pd.read_csv('Data/EconomicTracker-main/data/GeoIDs - City.csv')
# Rename cityname from Washington to Wasington DC to match to the czname
czone.loc[czone.cityname =='Washington', 'cityname'] = 'Washington DC'
# Drop unnecessary columns 
czone.drop(['cityid','stateabbrev', 'statename', 'city_pop2019', 'statefips'], axis=1, inplace=True)
# Rename columns 
czone.rename({'lat':'cz_lat', 'lon':'cz_lon', 'cityname':'nearest_city'}, axis=1, inplace=True)


# merge czone and county_zip
df = county_zip.merge(czone, how='left', right_on=['nearest_city'], left_on=['czname'])
df.head()

,countyfips,countyname,cityid,cityname,cz,czname,statename,statefips,stateabbrev,county_pop2019,...,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO,TOT_RATIO_x,TOT_RATIO_y,RES_RATIO_y,BUS_RATIO_y,nearest_city,cz_lat,cz_lon
0,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.000000,1.0,0.0,1.000000,1.000000,0.000000,1.0,New York City,40.71,-74.01
1,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.000000,1.0,1.0,1.000000,1.000000,0.000000,1.0,New York City,40.71,-74.01
2,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.994811,1.0,1.0,0.995644,0.995644,0.994811,1.0,New York City,40.71,-74.01
3,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,40.71,-74.01
4,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,40.71,-74.01


Because my focus is the housing price movement in a metropolitan area, I drop the zip codes outside of a metropolitan area. 

In [10]:
# Drop zipcode outside of the top 53 greater economic zones. 
df.dropna(axis=0, subset=['nearest_city'], inplace=True)
# Drop missing zipcodes 
df.dropna(axis=0, subset=['ZIP'], inplace=True)

df.drop_duplicates()


,countyfips,countyname,cityid,cityname,cz,czname,statename,statefips,stateabbrev,county_pop2019,...,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO,TOT_RATIO_x,TOT_RATIO_y,RES_RATIO_y,BUS_RATIO_y,nearest_city,cz_lat,cz_lon
0,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.000000,1.0,0.0,1.000000,1.000000,0.000000,1.0,New York City,40.71,-74.01
1,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.000000,1.0,1.0,1.000000,1.000000,0.000000,1.0,New York City,40.71,-74.01
2,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,0.994811,1.0,1.0,0.995644,0.995644,0.994811,1.0,New York City,40.71,-74.01
3,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,40.71,-74.01
4,36103.0,Suffolk,NaN,NaN,19400.0,New York City,New York,36.0,NY,1476601.0,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,40.71,-74.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39483,2198.0,Prince of Wales-Hyder,NaN,NaN,28900.0,Denver,Alaska,2.0,AK,6203.0,...,0.000000,0.0,1.0,1.000000,1.000000,0.000000,0.0,Denver,39.74,-104.99
39484,2198.0,Prince of Wales-Hyder,NaN,NaN,28900.0,Denver,Alaska,2.0,AK,6203.0,...,0.000000,0.0,1.0,1.000000,1.000000,0.000000,0.0,Denver,39.74,-104.99
39485,2198.0,Prince of Wales-Hyder,NaN,NaN,28900.0,Denver,Alaska,2.0,AK,6203.0,...,1.000000,0.0,1.0,1.000000,1.000000,1.000000,0.0,Denver,39.74,-104.99
39486,2198.0,Prince of Wales-Hyder,NaN,NaN,28900.0,Denver,Alaska,2.0,AK,6203.0,...,0.000000,0.0,1.0,1.000000,1.000000,0.000000,0.0,Denver,39.74,-104.99


# 3. Matching ZIP codes to latitude and longitude data

Using a data from http://data.nber.org/data/zip-code-distance-database.html, I assign latitude and longitude data for each zipcode. 

First, I read the lat&lon data from the webside and merge it to my ZIP code data. 

In [11]:
# Read latitude and longitude data from http://data.nber.org/data/zip-code-distance-database.html 

url = 'http://data.nber.org/distance/2016/gaz/zcta5/gaz2016zcta5centroid.csv'
lonlat = pd.read_csv(url)

# rename zcta5 (5digit zipcode) to ZIP to match with df
lonlat.rename({'zcta5':'ZIP'}, axis=1, inplace=True)

lonlat.drop_duplicates()

# merge lonlat and df 
df_lonlat = df.merge(lonlat, how='left', on='ZIP')

print('The number of zip codes which longitudes and latitudes are missing:', df_lonlat.isnull().sum()[-1:])

The number of zip codes which longitudes and latitudes are missing: intptlong    2861
dtype: int64


I use longitude and latitude to calculate the city center's distance and assign urban and suburban areas, which is a core indicator of my analysis and model. So, I drop if a zip code has no longitude/latitude information.

In [12]:
# Drop zipcodes with no longitudes and latitudes

df_lonlat.dropna(axis=0, subset=['intptlong'], inplace=True)

Lastly, for each zip code, I calcuate a distance to a city center of a nearest city.  
I calculate the distance using the Haversine Formula using longitude and latitude information of two points. 
In the cell below, I define Havesine Function and I calcuate the distance using this function. 

In [13]:
# Define Haversine Function 

def haversine_vectorize(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    newlon = lon2 - lon1
    newlat = lat2 - lat1

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km


I calcuate the distance using Havesine Function, and I dropped unnecessary columns. 

In [14]:
# Finding the distance between a zipcode and a city center using the Haversine Formula. 
df_lonlat['distance'] = haversine_vectorize(df_lonlat['cz_lon'],df_lonlat['cz_lat'],df_lonlat['intptlong'],
                   df_lonlat['intptlat'])

df_lonlat.drop_duplicates(inplace=True)

# Clearn up df_lonlat 

col_drop = ['cz', 'COUNTY','USPS_ZIP_PREF_STATE','cz_lat', 'cz_lon', 'intptlat', 'intptlong' ]

df_geo = df_lonlat.drop(col_drop, axis=1)

df_geo.drop_duplicates(inplace=True)
df_geo.head()


,countyfips,countyname,cityid,cityname,czname,statename,statefips,stateabbrev,county_pop2019,ZIP,USPS_ZIP_PREF_CITY,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO,TOT_RATIO_x,TOT_RATIO_y,RES_RATIO_y,BUS_RATIO_y,nearest_city,distance
1,36103.0,Suffolk,NaN,NaN,New York City,New York,36.0,NY,1476601.0,6390,FISHERS ISLAND,0.000000,1.0,1.0,1.000000,1.000000,0.000000,1.0,New York City,182.283194
2,36103.0,Suffolk,NaN,NaN,New York City,New York,36.0,NY,1476601.0,11701,AMITYVILLE,0.994811,1.0,1.0,0.995644,0.995644,0.994811,1.0,New York City,50.643368
3,36103.0,Suffolk,NaN,NaN,New York City,New York,36.0,NY,1476601.0,11702,BABYLON,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,63.361280
4,36103.0,Suffolk,NaN,NaN,New York City,New York,36.0,NY,1476601.0,11703,NORTH BABYLON,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,57.745033
5,36103.0,Suffolk,NaN,NaN,New York City,New York,36.0,NY,1476601.0,11704,WEST BABYLON,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,New York City,54.907553


## Now I have df_geo. It links county, state, zip codes, and has distance to the city center.  I save this in df_geo.csv 

In [15]:
df_geo.to_csv('Data/df_geo.csv', index = False)